# 📦 1. Setup Environment for LoRA/PEFT

This notebook covers the first step in our LoRA/PEFT checklist: setting up the environment.

## Checklist Items:
- ✅ Install Python >= 3.8
- ✅ Create virtual environment
- ✅ Install PyTorch (GPU support)
- ✅ Install required libraries
- ✅ Verify installation

## 1. Check Python Version

In [ ]:
import sys
print(f"Python version: {sys.version}")

# Check if Python >= 3.8
if sys.version_info >= (3, 8):
    print("✅ Python version is compatible")
else:
    print("❌ Python version is too old. Please upgrade to Python 3.8+")

## 2. Install Required Packages

Run the following commands in your terminal to install the required packages:

```bash
# Create virtual environment
python -m venv lora_env
source lora_env/bin/activate  # On Windows: lora_env\Scripts\activate

# Install packages
pip install -r requirements.txt
```

## 3. Verify Installation

In [ ]:
# Check PyTorch installation
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✅ CUDA version: {torch.version.cuda}")
        print(f"✅ GPU count: {torch.cuda.device_count()}")
except ImportError:
    print("❌ PyTorch not installed")

In [ ]:
# Check Transformers
try:
    import transformers
    print(f"✅ Transformers version: {transformers.__version__}")
except ImportError:
    print("❌ Transformers not installed")

In [ ]:
# Check PEFT
try:
    import peft
    print(f"✅ PEFT version: {peft.__version__}")
except ImportError:
    print("❌ PEFT not installed")

In [ ]:
# Check other important libraries
libraries = [
    ('datasets', 'datasets'),
    ('accelerate', 'accelerate'),
    ('bitsandbytes', 'bitsandbytes'),
    ('numpy', 'numpy'),
    ('scipy', 'scipy'),
    ('evaluate', 'evaluate'),
    ('tqdm', 'tqdm'),
]

for name, module in libraries:
    try:
        lib = __import__(module)
        version = getattr(lib, '__version__', 'unknown')
        print(f"✅ {name}: {version}")
    except ImportError:
        print(f"❌ {name}: not installed")

## 4. Test Basic Functionality

In [ ]:
# Test basic model loading
from transformers import AutoTokenizer, AutoModel

try:
    # Load a small model for testing
    model_name = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    print(f"✅ Successfully loaded {model_name}")
    print(f"✅ Model parameters: {model.num_parameters():,}")
    
    # Test tokenization
    text = "Hello, this is a test!"
    tokens = tokenizer(text, return_tensors="pt")
    print(f"✅ Tokenization works: {tokens['input_ids'].shape}")
    
    # Test model forward pass
    with torch.no_grad():
        outputs = model(**tokens)
    print(f"✅ Model forward pass works: {outputs.last_hidden_state.shape}")
    
except Exception as e:
    print(f"❌ Error testing basic functionality: {e}")

In [ ]:
# Test PEFT functionality
from peft import LoraConfig, get_peft_model, TaskType

try:
    # Create LoRA config
    lora_config = LoraConfig(
        task_type=TaskType.FEATURE_EXTRACTION,
        r=16,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["query", "value"]
    )
    
    # Apply LoRA to model
    peft_model = get_peft_model(model, lora_config)
    
    print(f"✅ PEFT model created successfully")
    
    # Print trainable parameters
    trainable_params = 0
    all_param = 0
    for _, param in peft_model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    
    print(f"✅ Trainable params: {trainable_params:,} ({100 * trainable_params / all_param:.2f}%)")
    print(f"✅ All params: {all_param:,}")
    
except Exception as e:
    print(f"❌ Error testing PEFT functionality: {e}")

## 5. Test Our Custom Implementation

In [ ]:
# Test our custom implementation
import sys
sys.path.append('..')  # Add parent directory to path

try:
    from config import ModelConfig, PEFTConfig
    from models import PEFTModelWrapper
    
    # Create configurations
    model_config = ModelConfig(
        model_name_or_path="distilbert-base-uncased",
        num_labels=2
    )
    
    peft_config = PEFTConfig(
        peft_type="LORA",
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=16
    )
    
    # Create model wrapper
    model_wrapper = PEFTModelWrapper(model_config, peft_config)
    peft_model = model_wrapper.load_model()
    
    print("✅ Custom implementation works!")
    print(f"✅ Model info: {model_wrapper.get_model_info()}")
    
except Exception as e:
    print(f"❌ Error testing custom implementation: {e}")
    import traceback
    traceback.print_exc()

## 6. Environment Summary

In [ ]:
import platform
import psutil

print("🖥️ System Information:")
print(f"Platform: {platform.platform()}")
print(f"Python: {sys.version.split()[0]}")
print(f"CPU cores: {psutil.cpu_count()}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")

if torch.cuda.is_available():
    print(f"\n🚀 GPU Information:")
    for i in range(torch.cuda.device_count()):
        gpu_name = torch.cuda.get_device_name(i)
        gpu_memory = torch.cuda.get_device_properties(i).total_memory / (1024**3)
        print(f"GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")
else:
    print("\n⚠️ No GPU available - training will be slower")

print("\n✅ Environment setup complete!")
print("\n📝 Next steps:")
print("1. Open notebook 02_data_preparation.ipynb")
print("2. Learn about data loading and preprocessing")
print("3. Prepare your dataset for training")